Pablo Jimeno - UPV/EHU 2016

In [ ]:
##################################################################################################################
### WISE utilities:
##################################################################################################################


def download_wise_fits(img_id, img_ra, img_dec, img_arcsec, wise_band):
    """ 
    fits file info: ( hdu_list.info() )
    No.    Name         Type      Cards   Dimensions   Format
    0    PRIMARY     PrimaryHDU      67   (XXX, XXX)   float32 
    """
           
    fits_file = '%s/FITS/IMG_%06d_WISE,_%04d_arcsec,_band_%i.fits' % (raw_dir, img_id, img_arcsec, wise_band)
    temp_dir = '%s/Temp_files' % data_dir
    
    count = 0
    while (not os.path.isfile(fits_file)) and (count < 50):
        count += 1
        try:
            base_url = 'http://irsa.ipac.caltech.edu/ibe/search/wise/allwise/p3am_cdd?columns=coadd_id,'
            query_band = 'band&where=band%20IN%20' + '(%i)' % (wise_band)
            query_pos = '&POS=%.4f,%.4f&INTERSECT=CENTER&mcen' % (img_ra, img_dec)
            final_url = base_url + query_band + query_pos
            print 'Query URL:\n', final_url
            urllib.urlretrieve(final_url, '%s/query_url.dat' % temp_dir)
            print '\nData received:' 
            !cat '$temp_dir/query_url.dat'
            !tail -n 1 '$temp_dir/query_url.dat' > '$temp_dir/query_data.dat'
            meta_data = np.genfromtxt('%s/query_data.dat' % temp_dir, dtype='str')

            params = {'coadd_id':meta_data[-2] , 'band':wise_band}
            params['coaddgrp'] = params['coadd_id'][:2]
            params['coadd_ra'] = params['coadd_id'][:4]

            path = str.format('{coaddgrp:s}/{coadd_ra:s}/{coadd_id:s}/{coadd_id:s}-w{band:1d}-int-3.fits?',**params)
            cutoff_url = 'center=%.4f,%.4fdeg&size=%i,%iarcsec&gzip=false' % (img_ra, img_dec, img_arcsec, img_arcsec)

            img_url = 'http://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/' + path + cutoff_url

            print '\nDownloading fits image from:\n%s' % img_url 
            urllib.urlretrieve(img_url, fits_file)
            print '\nFits image saved in:\n%s\n\n******************************\n' % fits_file
        except:
            pass
            


def get_wise_image_data(img_id, img_arcsec, wise_band):

    fits_file = '%s/WISE/FITS/IMG_%06d_WISE,_%04d_arcsec,_band_%i.fits' % (raw_dir, img_id, img_arcsec, wise_band)
    img_data = fits.getdata(fits_file)

    img_data = np.rot90(np.rot90(np.fliplr(img_data)))
    img_data = np.array(img_data, dtype=np.float)
        
    return img_data



def wise_fits_to_png(img_id, img_arcsec, img_pix, wise_band, circles=False, **circles_kwargs):
               
    png_file  = '%s/WISE/PNG/IMG_%06d_WISE,_%04d_arcsec,_band_%i.png' % (raw_dir, img_id, img_arcsec, wise_band)

    img_data = get_wise_image_data(img_id, img_arcsec, wise_band)

    if not circles:
        fig = plot_image_data(img_data)
    elif circles:
        fig = plot_image_data_circles(img_data, img_arcsec, circles_kwargs['r_bins'])
    
    fig.savefig(png_file)
    
    
    
    
##################################################################################################################
### Planck utilities:
##################################################################################################################


def read_planck_map(map_file, display=False):
    
    fits_file = '%s/Planck/FITS_maps/%s' % (raw_dir, map_file)
    planck_map = hp.read_map(fits_file)
    Nside = np.int(np.sqrt(planck_map.size/12.))

    if display:
        hp.mollview(planck_map, nest=False, title=map_file)
        
    return planck_map, Nside

 

def get_planck_patch(planck_map, clu_id, img_pix, zoom_factor):
    
    Nside = np.int(np.sqrt(planck_map.size/12.))
    
    clu = Cluster(redm, clu_id)
    clu_long = clu.l.radian # l: gal longitude, in radians
    clu_lat = clu.b.radian # b: gal latitude, in radians
        
    map2D = np.zeros((img_pix, img_pix))
    
    ix, iy = np.ogrid[0:img_pix, 0:img_pix]

    X = 2.2/zoom_factor*(ix - img_pix/2.)/img_pix
    Y = 2.2/zoom_factor*(iy - img_pix/2.)/img_pix
    rho = np.sqrt(X**2 + Y**2) + 10.**(-9)

    C = np.arcsin(rho)
    Arg1 = np.cos(C)*np.sin(clu_lat)
    Arg2 = (Y*np.sin(C)*np.cos(clu_lat))/rho
    Arg3 = rho*np.cos(clu_lat)*np.cos(C) - Y*np.sin(clu_lat)*np.sin(C)
    Arg4 = X*np.sin(C)
    Arg5 = Arg4/Arg3

    pix_lat  = np.arcsin(Arg1 + Arg2)
    pix_long = clu_long + np.arctan(Arg5)

    fix_mask = Arg3 <= 0
    pix_long[fix_mask] = np.pi + pix_long[fix_mask]

    theta_ring = np.pi/2.0 - pix_lat
    phi_ring = pix_long

    iring = ang2pix(Nside, theta_ring, phi_ring, nest=False)
    map2D = planck_map[iring]

    StartPhi = pix_lat[img_pix/2,0]*(180.0/np.pi)
    EndPhi = pix_lat[img_pix/2,-1]*(180.0/np.pi)
    FOV_deg = np.abs(EndPhi - StartPhi)
    
    map2D = np.array(map2D, dtype=np.float16)
    return map2D, FOV_deg


   
    
def cluster_patch(clu_id, band, zoom_factor):
        
    patch_file = '%s/Planck_patchs/%s/Patch_%06d_%02d.npy' % (data_dir, band, clu_id, zoom_factor)
    if os.path.isfile(patch_file):
        map2D = np.load(patch_file)
        map2D = np.array(map2D, dtype=np.float)
        return map2D
    elif not os.path.isfile(patch_file):
        print '***ERROR: patch not found...***'
    
    
    

##################################################################################################################
### SDSS, Planck & ROSAT utilities:
##################################################################################################################
    
    
#def download_sdss(img_dir, img_redm_id, img_ra, img_dec, img_scale, img_arcsec, img_pix_size):
#    
#    base_url = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx?'
#    query_url = 'ra=%.10f&dec=%.10f&scale=%.4f&width=%i&height=%i' % (img_ra, img_dec, img_scale, img_pix_size, img_pix_size)
#    img_url = base_url + query_url
#    urllib.urlretrieve(img_url, '%s/SDSS/IMG_%06d_SDSS,_%i_arcsec.png' % (img_dir, img_redm_id, img_arcsec))
    
    
    
#def process_planck(img_dir, img_id, img_pix_size):
#    
#    raw_file  = '%s/Planck/RAW/SZ_%06d' % (img_dir, img_id)
#    png_file  = '%s/Planck/PNG/IMG_%06d.png' % (img_dir, img_id)
#    
#    if os.path.isfile(raw_file):
#        img = Image.open(raw_file)
#        img = img.resize((img_pix_size, img_pix_size), Image.ANTIALIAS)
#        img.save(png_file)
    

    
#def process_rosat(img_dir, img_id, img_pix_size):
#
#    raw_file  = '%s/ROSAT/RAW/ROSAT_R6_%06d.gif' % (img_dir, img_id)
#    png_file  = '%s/ROSAT/PNG/IMG_ROSAT_R6_%06d.png' % (img_dir, img_id)
#    
#    if os.path.isfile(raw_file):
#        img = Image.open(raw_file)
#        img = img.resize((img_pix_size, img_pix_size), Image.ANTIALIAS)
#        img.save(png_file)
        
    
    
##################################################################################################################
### WISE flux & light analysis:
##################################################################################################################


def radial_binning(min_arcsec, max_arcsec, nbins):
    
    r_bins = np.linspace(min_arcsec, max_arcsec, nbins+1)
    
    return r_bins



def compute_flux_distri(img_data, img_arcsec, r_bins):
    
    arcsecs_in_pix = img_arcsec/img_data.shape[0]
    
    lx, ly = img_data.shape
    X, Y = np.ogrid[0:lx, 0:ly]

    dist_matrix = distances_image(img_pix)*arcsecs_in_pix ## distances from center in arcsecs
        
    fluxes = []
    distances = (r_bins[:-1] + r_bins[1:])/2.
    
    for n in range(len(r_bins)-1):
        r_min = r_bins[n]
        r_max = r_bins[n+1]
        radial_mask = (dist_matrix >= r_min) & (dist_matrix < r_max)
        radial_area = np.pi*(r_max**2-r_min**2)
        flux = img_data[radial_mask].sum()/radial_area
        fluxes.append(flux)

    return np.array(fluxes)



def compute_light_distri(img_data, img_arcsec, r_bins):
    
    arcsecs_in_pix = img_arcsec/image_data.shape[0]
    
    lx, ly = img_data.shape
    X, Y = np.ogrid[0:lx, 0:ly]

    dist_matrix = np.sqrt((X - lx/2)**2 + (Y - ly/2)**2)*arcsecs_in_pix ## distances from center in arcsecs
        
    lights = []
    distances = (r_bins[:-1] + r_bins[1:])/2.
    
    for n in range(len(r_bins)-1):
        r_max = r_bins[n+1]
        radial_mask = (dist_matrix < r_max)
        radial_area = np.pi*(r_max**2)
        light = img_data[radial_mask].sum()/radial_area
        lights.append(light)

    return np.array(lights)



def plot_flux_distribution(out_dir, img_redm_id, wise_bands, fluxes, cent_fit, back_fit, r_bins):
    """Flux of cluster"""
    
    out_file = '%s/FLUX/%06d_flux.png' % (out_dir, img_redm_id)
    
    distances = (r_bins[:-1] + r_bins[1:])/2.
    dist_fit = np.arange(distances[0], distances[-1], 1.)

    colors = ['red', 'black', 'blue']
    freqs = ['3.4', '4.6', '12', '22']
        
    figsizex, figsizey = 6.5, 4
    fig = plt.figure(figsize=(figsizex, figsizey))
    ax = fig.add_subplot(1, 1, 1)
    
    for band in range(len(wise_bands)):
        plt.plot(distances, fluxes[band], color=colors[band], label='W%i: %s $\mu$m' % (wise_bands[band], freqs[band]))
        
    plt.plot(dist_fit, back_fit + func_fit(dist_fit, *cent_fit), 'k', label='fit flux')
    plt.plot(dist_fit, np.full(len(dist_fit), back_fit), 'g:', label='fit background', linewidth=3)
    plt.plot(dist_fit, func_fit(dist_fit, *cent_fit), 'b:', label='fit exp', linewidth=3)
        
    plt.grid(alpha=0.5)
    plt.xlabel(r'r [arcsecs]')
    plt.ylabel(r'flux [DI arsec$^{-2}$]')
    ax.set_xlim([r_bins[0], r_bins[-1]])
    #ax.set_xscale('log')
    #ax.set_yscale('log')
    
    ax.set_ylim([0.,6.])
    ax.set_xlim([0.,120.])
    
    plt.legend(loc='upper right', frameon=False, labelspacing=0.5)

    fig.savefig(out_file)
    plt.close()
    
    
    
    
def plot_light_distribution(out_dir, img_redm_id, wise_bands, lights, cent_fit, back_fit, fit_light, half_light, r_bins):
    """Flux of cluster"""
    
    out_file = '%s/LIGHT/%06d_light.png' % (out_dir, img_redm_id)
    
    distances = (r_bins[:-1] + r_bins[1:])/2.
    dist_fit = np.arange(distances[0], distances[-1], 0.1)

    colors = ['red', 'black', 'blue']
    freqs = ['3.4', '4.6', '12', '22']
        
    figsizex, figsizey = 6.5, 4
    fig = plt.figure(figsize=(figsizex, figsizey))
    ax = fig.add_subplot(1, 1, 1)
    
    for band in range(len(wise_bands)):
        plt.plot(distances, lights[band], color=colors[band], label='W%i: %s $\mu$m' % (wise_bands[band], freqs[band]))
    
    plt.plot(dist_fit, fit_light, 'k', label='fit flux')
    plt.plot(dist_fit, np.full(len(dist_fit), back_fit), 'g:', label='fit background', linewidth=3)
    plt.plot([half_light, half_light], [0., 10000.], 'b', label='half-light', linewidth=1)
        
    plt.grid(alpha=0.5)
    plt.xlabel(r'r [arcsecs]')
    plt.ylabel(r'Surface brightness [DI arcecs$^{-2}$]')
    ax.set_xlim([r_bins[0], r_bins[-1]])
    #ax.set_xscale('log')
    #ax.set_yscale('log')
    
    ax.set_ylim([0.,6.])
    ax.set_xlim([0.,120.])
    
    plt.legend(loc='upper right', frameon=False, labelspacing=0.5)

    fig.savefig(out_file)
    plt.close()



def func_fit(x, *p):
    a, b = p
    return a*np.exp(-x/b)
    
    

def fit_flux_distri(fluxes, r_bins):
    
    distances = (r_bins[:-1] + r_bins[1:])/2.
    
    back_mask = distances >= 80.   
    back_fit  = np.median(fluxes[back_mask])

    err = lambda p: np.mean((func_fit(distances,*p)-(fluxes-back_fit))**2)
    p_init = [1., 25]
    cent_fit = minimize(err, p_init, bounds=[(0., None),(0., None)], method="L-BFGS-B").x
    chi2 = err(cent_fit)
    #p_opt = curve_fit(func, distances, fluxes, maxfev=100000)[0]    

    return cent_fit, back_fit, chi2




def check_light_distri(lights, cent_fit, back_fit, r_bins):
    
    distances = (r_bins[:-1] + r_bins[1:])/2.
    
    dist_fit = np.arange(distances[0], distances[-1], 0.1)

    total_light = []
    total_back_light = []
    
    for d in range(len(dist_fit)):
        bin_light = integrate.quad(lambda x: (back_fit + func_fit(x,*cent_fit))*2*np.pi*x, 0., dist_fit[d])[0]
        bin_back_light = integrate.quad(lambda x: (back_fit)*2*np.pi*x, 0., dist_fit[d])[0]
        total_light.append(bin_light/(np.pi*dist_fit[d]**2))
        total_back_light.append(bin_back_light/(np.pi*dist_fit[d]**2))
    
    fit_light = np.array(total_light)
    
    sum_light = 0.
    for d in range(len(dist_fit)):
        sum_light += fit_light[d]
        if (sum_light > (fit_light.sum() - np.array(total_back_light).sum())/2.):
            half_light = dist_fit[d]
            break
            
    return fit_light, half_light



def process_half_light(z, angle_arcsecs):
    
    arcsecs_in_rad = 360.*3600./(2.*np.pi)
    ad_dist = Planck13.angular_diameter_distance(z).value
    physical_transverse_dist = ad_dist*(angle_arcsecs/arcsecs_in_rad)
       
    return physical_transverse_dist
    